In [1]:
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import seaborn as sns
import shutil

from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

In [2]:
train_path = "/Users/sailaharimullapudi/Desktop/kneeKL224/train"
test_path = "/Users/sailaharimullapudi/Desktop/kneeKL224/test"
valid_path = "/Users/sailaharimullapudi/Desktop/kneeKL224/val"
list_of_classes = ['Healthy', 'Doubtful', 'Minimal', 'Moderate', 'Severe']

import os
import pandas as pd

for d in [train_path, test_path, valid_path]:
    filepaths = []
    labels = [] 
    classlist = os.listdir(d)
    
    for klass in classlist:
        try:
            intklass = int(klass)
            label = list_of_classes[intklass]
            classpath = os.path.join(d, klass)
            flist = os.listdir(classpath)        
            
            for f in flist:
                fpath = os.path.join(classpath, f)
                filepaths.append(fpath)
                labels.append(label)
        
        except ValueError:
            # Skip directories that are not valid integers
            continue
    
    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')        
    pdf = pd.concat([Fseries, Lseries], axis=1)
    
    if d == test_path:
        test_df = pdf
    elif d == valid_path:
        valid_df = pdf
    else:
        train_df = pdf

print('train_df length: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))

# get the number of classes and the images count for each class in train_df
classes = sorted(list(train_df['labels'].unique()))
class_count = len(classes)
print('The number of classes in the dataset is: ', class_count)
groups = train_df.groupby('labels')
print('{0:^30s} {1:^13s}'.format('CLASS', 'IMAGE COUNT'))
countlist = []
classlist = []
for label in sorted(list(train_df['labels'].unique())):
    group = groups.get_group(label)
    countlist.append(len(group))
    classlist.append(label)
    print('{0:^30s} {1:^13s}'.format(label, str(len(group))))

# lets get the average height and width of a sample of the train images
ht = 0
wt = 0

train_df_sample = train_df.sample(n = 100, random_state = 123, axis = 0)
for i in range (len(train_df_sample)):
    fpath = train_df_sample['filepaths'].iloc[i]
    img = plt.imread(fpath)
    shape = img.shape
    ht += shape[0]
    wt += shape[1]

train_df length:  5778   test_df length:  1656   valid_df length:  826
The number of classes in the dataset is:  5
            CLASS               IMAGE COUNT 
           Doubtful                1046     
           Healthy                 2286     
           Minimal                 1516     
           Moderate                 757     
            Severe                  173     


In [3]:
classes = sorted(list(test_df['labels'].unique()))
class_count = len(classes)
print('The number of classes in the dataset is: ', class_count)
groups = test_df.groupby('labels')
print('{0:^30s} {1:^13s}'.format('CLASS', 'IMAGE COUNT'))
countlist = []
classlist = []
for label in sorted(list(test_df['labels'].unique())):
    group = groups.get_group(label)
    countlist.append(len(group))
    classlist.append(label)
    print('{0:^30s} {1:^13s}'.format(label, str(len(group))))

The number of classes in the dataset is:  5
            CLASS               IMAGE COUNT 
           Doubtful                 296     
           Healthy                  639     
           Minimal                  447     
           Moderate                 223     
            Severe                  51      


In [4]:
# image size - avg hight and width
average_height = ht // 100
average_width = wt // 100
image_size = (average_height, average_width)

# preprocess images
def preprocess(filepaths):
    images = []
    for filepath in filepaths:
        img = plt.imread(filepath)
        img = resize(img, image_size, anti_aliasing=True)
        img_flat = img.flatten()
        images.append(img_flat)
    return images

In [5]:
train_images = preprocess(train_df['filepaths'])
test_images = preprocess(test_df['filepaths'])
valid_images = preprocess(valid_df['filepaths'])

In [6]:
# label encoding 
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df['labels'])
test_labels = label_encoder.transform(test_df['labels'])
valid_labels = label_encoder.transform(valid_df['labels'])

print(train_labels)

[1 1 1 ... 2 2 2]


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

all_data_labels = np.concatenate([train_labels, valid_labels])
all_data_images = np.concatenate([train_images, valid_images])

kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

svm = SVC(kernel = 'linear', probability = True, decision_function_shape = 'ovr', C = 1.0)

cross_val = cross_val_score(svm, all_data_images, all_data_labels, cv = kfold, scoring = 'accuracy')

svm.fit(all_data_images, all_data_labels)

test_predictions = svm.predict(test_images)
accuracy = accuracy_score(test_labels, test_predictions)
report = classification_report(test_labels, test_predictions, target_names=list_of_classes)

print(f"Cross-Validation Accuracy: {np.mean(cross_val)}")
print(f"Test Set Accuracy: {accuracy}")
print("Classification Report:\n", report)

In [ ]:
matrix = confusion_matrix(test_labels, test_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, cmap = "Blues", fmt="d", cbar=False,
            xticklabels=list_of_classes, yticklabels=list_of_classes)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
decision_function = svm.decision_function(test_images)
train_labels_binary = label_binarize(test_labels, classes=np.unique(train_labels))
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(np.unique(train_labels))):
    fpr[i], tpr[i], _ = roc_curve(train_labels_binary[:, i], decision_function[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
plt.figure(figsize=(10, 6))
for i in range(len(np.unique(train_labels))):
    x = ""
    if i == 0:
        x = "Healthy"
    elif i == 1:
        x = "Doubtful"
    elif i == 2:
        x = "Minimal"
    elif i == 3:
        x = "Moderate"
    else:
        x = "Severe"
    plt.plot(fpr[i], tpr[i], lw=2, label='ROC Curve: Class {} (AUC = {:.2f})'.format(x, roc_auc[i]))

plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class (Support Vector Machine)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix_heatmap(data, class_names, title, xlabel, ylabel):
    df_cm = pd.DataFrame(data, index=[xlabel], columns=class_names)
    plt.figure(figsize=(4, 4))
    sns.heatmap(df_cm, annot=True, fmt='g', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

conf_matrix = confusion_matrix(test_labels, test_predictions)

sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = np.diagonal(conf_matrix[::-1, ::-1]) / conf_matrix[::-1, ::-1].sum(axis=1)

sensitivity_heatmap = np.array([sensitivity])
specificity_heatmap = np.array([specificity])

plot_confusion_matrix_heatmap(sensitivity_heatmap, list_of_classes, title='Sensitivity', xlabel='', ylabel='Sensitivity')

plot_confusion_matrix_heatmap(specificity_heatmap, list_of_classes, title='Specificity', xlabel='', ylabel='Specificity')